In [1]:
import pandas as pd

In [2]:
%store -r gdf_annual_with_svi
%store -r gdf_quarter_with_svi
%store -r gdf_semi_with_svi

gdf_annual = gdf_annual_with_svi

In [3]:
gdf_annual.columns

Index(['geoid', 'year', 'deaths', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'NAME',
       'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'lat', 'lon',
       'geometry', 'timestep', 'theme_1_pc', 'theme_2_pc', 'theme_3_pc',
       'theme_4_pc', 'svi_pctile'],
      dtype='object')

In [4]:
gdf_annual

,geoid,year,deaths,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,...,AWATER,lat,lon,geometry,timestep,theme_1_pc,theme_2_pc,theme_3_pc,theme_4_pc,svi_pctile
5637,17031010100,2014,0,17,031,010100,101,Census Tract 101,G5020,S,...,0.0,+42.0212553,-087.6698301,POINT (-87.67202 42.01989),1,0.6378,0.2641,0.6045,0.9189,0.7026
0,17031010100,2015,1,17,031,010100,101,Census Tract 101,G5020,S,...,0.0,+42.0212553,-087.6698301,POINT (-87.67202 42.01989),2,0.7266,0.3552,0.6652,0.8699,0.7562
1,17031010100,2016,3,17,031,010100,101,Census Tract 101,G5020,S,...,0.0,+42.0212553,-087.6698301,POINT (-87.67142 42.02143),3,0.7266,0.3552,0.6652,0.8699,0.7562
2,17031010100,2017,2,17,031,010100,101,Census Tract 101,G5020,S,...,0.0,+42.0212553,-087.6698301,POINT (-87.66805 42.02216),4,0.6993,0.3324,0.6231,0.9294,0.7652
5636,17031010100,2018,0,17,031,010100,101,Census Tract 101,G5020,S,...,0.0,+42.0212553,-087.6698301,POINT (-87.67202 42.01989),5,0.6993,0.3324,0.6231,0.9294,0.7652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5621,17031844700,2018,6,17,031,844700,8447,Census Tract 8447,G5020,S,...,0.0,+41.8541473,-087.7117002,POINT (-87.71600 41.85288),5,0.9499,0.6778,0.9103,0.9653,0.9760
5622,17031844700,2019,3,17,031,844700,8447,Census Tract 8447,G5020,S,...,0.0,+41.8541473,-087.7117002,POINT (-87.70994 41.85512),6,0.9499,0.6778,0.9103,0.9653,0.9760
5623,17031844700,2020,5,17,031,844700,8447,Census Tract 8447,G5020,S,...,0.0,+41.8541473,-087.7117002,POINT (-87.70690 41.85560),7,0.9499,0.6778,0.9103,0.9653,0.9760
5624,17031844700,2021,4,17,031,844700,8447,Census Tract 8447,G5020,S,...,0.0,+41.8541473,-087.7117002,POINT (-87.71146 41.85312),8,0.9499,0.6778,0.9103,0.9653,0.9760


In [15]:
geoid_data = []

for geoid in gdf_annual['geoid'].unique():
    geoid_df = gdf_annual[gdf_annual['geoid'] == geoid]
    
    # Extract the deaths for lookback years
    deaths_2020 = geoid_df[geoid_df['year'] == 2020]['deaths'].values[0]
    deaths_2019 = geoid_df[geoid_df['year'] == 2019]['deaths'].values[0]
    deaths_2018 = geoid_df[geoid_df['year'] == 2018]['deaths'].values[0]
    
    # Extract the lat and lon values 
    lat = geoid_df['lat'].values[0]
    lon = geoid_df['lon'].values[0]
    
    # append data
    geoid_data.append([geoid, deaths_2020, deaths_2019, deaths_2018, lat, lon])


static_features = pd.DataFrame(geoid_data)
static_features = static_features.transpose()
file_path = '/Users/jyontika/Desktop/hughes-data/cook-county/static_features.csv'
static_features.to_csv(file_path, index=False, header=False)



In [17]:
geoid_series = gdf_annual['geoid']
file_path = '/Users/jyontika/Desktop/hughes-data/cook-county/locations.txt'
with open(file_path, 'w') as file:
    for geoid in geoid_series:
        file.write(str(geoid) + '\n')


In [ ]:
##SVI  dynamic features organizations


#take a subset
#take my df and turn into numpy array
#pickle.dump